In [1]:
import sys
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt

sys.path.append("..")
from hoinetx.communities.hy_mmsbm.model import HyMMSBM
from hoinetx.core.hypergraph import Hypergraph
from hoinetx.generation.hy_mmsbm_sampling import HyMMSBMSampler
from hoinetx.linalg.linalg import *
from hoinetx.dynamics.randwalk import *

np.random.seed(123)

In [119]:
# generate list of list of nodes from 0 to 10 with random length from 2 to 5, all nodes in a list must be different and all lists inside the list must be unique
def generate_random_hyperedges_list(num_hyperedges, num_nodes, min_hyperedge_size, max_hyperedge_size):
    hyperedges_list = []
    for i in range(num_hyperedges):
        hyperedge = np.random.choice(num_nodes, np.random.randint(min_hyperedge_size, max_hyperedge_size + 1), replace=False)
        hyperedges_list.append(hyperedge)
    hyperedges_list = list(set(map(tuple, hyperedges_list)))
    return hyperedges_list

edges_list = generate_random_hyperedges_list(200, 40, 2, 5)

#edges_list = list(set(edges_list))

In [120]:
print(f'Number of hlinks: {len(edges_list) }')
#check if all tuples are unique
print(len(edges_list) == len(set(map(tuple, edges_list))))
# check if all tuples in edge_list have unique elements
print(all(len(set(edge)) == len(edge) for edge in edges_list))


Number of hlinks: 200
True
True


In [121]:
HG = Hypergraph(edges_list)

In [5]:
K = transition_matrix(HG).todense()

In [6]:
# check if K is stochastic
print(np.allclose(np.sum(K, axis=0), np.ones(K.shape[0])))


True


In [108]:
# define a function called random_walk that given an object HG of this type hoinetx.core.hypergraph.Hypergraph and a starting node s returns a list of nodes visited by the random walk
def random_walk(HG : Hypergraph, s : int, time : int)->list:
    '''Compute the random walk on the hypergraph.
    
    Parameters
    ----------
    HG : Hypergraph
        The hypergraph on which the random walk is defined.
    s : int
        The starting node of the random walk.
        
    Returns
    -------
    nodes : list
        The list of nodes visited by the random walk.
    '''
    K = np.array(transition_matrix(HG).todense() )
    nodes = [s]
    for t in range(time):
        next_node = np.random.choice(K.shape[0], p=K[:, nodes[-1]])
        nodes.append(next_node)
    return nodes



In [122]:
# find the stationary state of the random walk
def RW_stationary_state(HG : Hypergraph)->np.ndarray:
    '''Compute the stationary state of the random walk on the hypergraph.
    
    Parameters
    ----------
    HG : Hypergraph
        The hypergraph on which the random walk is defined.
        
    Returns
    -------
    stationary_state : np.ndarray
        The stationary state of the random walk on the hypergraph.
    '''
    K = np.array(transition_matrix(HG).todense() )
    stationary_state = np.linalg.solve(np.eye(K.shape[0]) - K.T, np.ones(K.shape[0]))
    stationary_state = stationary_state / np.sum(stationary_state)
    return stationary_state


In [123]:
RW_stationary_state(HG)


array([0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
       0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
       0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
       0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
       0.025, 0.025, 0.025, 0.025])

In [118]:
HG.num_nodes()

10

In [26]:
K[:, nodes[-1]]

matrix([[0.2459893 ],
        [0.05614973],
        [0.06951872],
        [0.0855615 ],
        [0.10160428],
        [0.05882353],
        [0.0828877 ],
        [0.09625668],
        [0.12032086],
        [0.0828877 ]])

In [28]:
nodes

[0, 8, 1]